In [34]:
!pip install nltk
!pip install transformers
!pip install tf-keras

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
    --------------------------------------- 0.0/1.7 MB 660.6 kB/s eta 0:00:03
   - -------------------------------------- 0.1/1.7 MB 812.7 kB/s eta 0:00:03
   -- ------------------------------------- 0.1/1.7 MB 819.2 kB/s eta 0:00:02
   --- ------------------------------------ 0.1/1.7 MB 853.3 kB/s eta 0:00:02
   ---- ----------------------------------- 0.2/1.7 MB 908.0 kB/s eta 0:00:02
   ----- ---------------------------------- 0.2/1.7 MB 901.1 kB/s eta 0:00:02
   ------ --------------------------------- 0.3/1.7 MB 930.9 kB/s eta 0:00:02
   ------- -------------------------------- 0.3/1.7 MB 952.6 kB/s eta 0:00:02
   --------- ------------------------------ 0.4/1.7 MB 969.0 kB/s eta 0:00:02
   --------- ------------------------------ 0.4/1.7 MB 970.6 kB/s eta 0:00:02
   ---------- ----------------------------- 0.5/1.7 MB 982.5 kB/s eta 0:00:02
   ------------ --------------------------- 0.5/1.7 MB 992.2 kB/s eta 0:0

In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import json
import nltk
import warnings
from sklearn.model_selection import train_test_split

### For preprocessing of the data

In [29]:
with open('comments.json', 'r',encoding='utf-8') as f:
    comments_data = json.load(f)

# Create a DataFrame
comments = []
for item in comments_data['items']:
    comment = item['snippet']['topLevelComment']['snippet']['textOriginal']
    comments.append(comment)

# Print out the comments
#for i, comment in enumerate(comments, 1):
#    print(f"Comment {i}: {comment}")

comments_df = pd.DataFrame(comments, columns=['comment'])

#Preprocess data
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r'\@\w+|\#', '', text)  # Remove mentions and hashtags
    text = text.lower()  # Convert to lowercase
    return text

comments_df['preprocessed_text'] = comments_df['comment'].apply(preprocess_text)

# Print out the comments
for i, comment in enumerate(comments_df['preprocessed_text'], 1):
    print(f"Comment {i}: {comment}")

# Split data into training and validation sets
train_texts, val_texts = train_test_split(comments_df['preprocessed_text'], test_size=0.2, random_state=42)

Comment 1: at 9:13, i made a mistake. sugar is mentioned in the label. but hidden sugar is not mentioned, which is why the total sugar on the label is incorrect. read this report for the details  🚨 unacademy manzil 2024 - the biggest event for upsc | join me live on unacademy unstoppable event link:  block your calendar | 28 april | 01 pm
Comment 2: it is the responsibility of the indian government, not the multinational corporations.
Comment 3: superpower😂😂😂
Comment 4: nestle ki sales ko km karne ke liye me aaj se iske product nahi kharidunga 😮😮 kon kon sahmat hai ❤❤
Comment 5: one more thing-hrithik roshan also promotes paan masala (signature paan masala)
Comment 6: i feel there is one more way which we can use,the indian *spices which are approved in eu or singapore, we should purchase those spices only in india, you might be wondering how would you know that, well there is a ecommerce platform in singapore or eu countries you can search the brand and spice there and then order the 

In [46]:
# Suppress the specific FutureWarning from huggingface_hub
warnings.filterwarnings("ignore", category=FutureWarning, message="`resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.")

# Load the sentiment analysis pipeline with a specific model
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
sentiment_pipeline = pipeline("sentiment-analysis", model=model_name)

# Add sentiment scores to comments
def get_sentiment_score(text):
    try:
        result = sentiment_pipeline(text)[0]
        return result['score'] if result['label'] == 'POSITIVE' else -result['score']
    except Exception as e:
        print(f"Error processing text: {text}\n{e}")
        return 0.0

comments_df['sentiment_score'] = comments_df['preprocessed_text'].apply(get_sentiment_score)

# Normalize scores to range [-1, 1]
max_score = comments_df['sentiment_score'].max()
min_score = comments_df['sentiment_score'].min()
comments_df['normalized_score'] = comments_df['sentiment_score'].apply(lambda x: 2 * (x - min_score) / (max_score - min_score) - 1)

# Print out comments with sentiment scores
for i, row in comments_df.iterrows():
    print(f"Comment {i+1}: {row['comment']} - Sentiment Score: \033[1m{row['normalized_score']}\033[0m")

# Split data into training and validation sets
train_texts, val_texts = train_test_split(comments_df['preprocessed_text'], test_size=0.2, random_state=42)

Comment 1: At 9:13, I made a mistake. Sugar is mentioned in the label. But Hidden sugar is not mentioned, which is why the total sugar on the label is incorrect. Read this report for the details
https://stories.publiceye.ch/nestle-babies/

🚨 Unacademy Manzil 2024 - The Biggest Event for UPSC | Join Me LIVE on Unacademy Unstoppable  Event Link: https://www.youtube.com/watch?v=2tzmZ-3Ne3c
 Block Your Calendar | 28 April | 01 PM - Sentiment Score: -0.9968315814785439
Comment 2: It is the responsibility of the Indian government, not the multinational corporations. - Sentiment Score: 0.9814567004909716
Comment 3: SuperPower😂😂😂 - Sentiment Score: -0.6972606417551324
Comment 4: Nestle ki sales ko km karne ke liye me Aaj se iske product nahi kharidunga 😮😮
Kon kon sahmat hai ❤❤ - Sentiment Score: -0.9947967636906284
Comment 5: One more thing-Hrithik Roshan also promotes Paan Masala (Signature Paan Masala) - Sentiment Score: 0.9835700923397237
Comment 6: I feel there is one more way which  we ca

In [57]:
import json
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from transformers import pipeline
import warnings
from collections import Counter

# Suppress the specific FutureWarning from huggingface_hub
warnings.filterwarnings("ignore", category=FutureWarning, message="`resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.")

# Load comments from JSON file
with open('comments.json', 'r', encoding='utf-8') as f:
    comments_data = json.load(f)

# Create a DataFrame
comments = []
for item in comments_data['items']:
    comment = item['snippet']['topLevelComment']['snippet']['textOriginal']
    like_count = item['snippet']['topLevelComment']['snippet'].get('likeCount', 0)  # Use get to handle missing 'likeCount'
    comments.append({'comment': comment, 'like_count': like_count})

comments_df = pd.DataFrame(comments)

# Preprocess data
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r'\@\w+|\#', '', text)  # Remove mentions and hashtags
    text = text.lower()  # Convert to lowercase
    return text

comments_df['preprocessed_text'] = comments_df['comment'].apply(preprocess_text)

# Load the sentiment analysis pipeline with a specific model
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
sentiment_pipeline = pipeline("sentiment-analysis", model=model_name)

# Add sentiment scores to comments
def get_sentiment_score(text):
    try:
        result = sentiment_pipeline(text)[0]
        return result['score'] if result['label'] == 'POSITIVE' else -result['score']
    except Exception as e:
        print(f"Error processing text: {text}\n{e}")
        return 0.0

comments_df['sentiment_score'] = comments_df['preprocessed_text'].apply(get_sentiment_score)

# Normalize scores to range [-1, 1]
max_score = comments_df['sentiment_score'].max()
min_score = comments_df['sentiment_score'].min()
comments_df['normalized_score'] = comments_df['sentiment_score'].apply(lambda x: 2 * (x - min_score) / (max_score - min_score) - 1)

# Classify comments into positive and negative
comments_df['sentiment'] = comments_df['normalized_score'].apply(lambda x: 'positive' if x > 0 else 'negative')

# Calculate percentage of positive and negative comments
total_comments = len(comments_df)
positive_comments_count = len(comments_df[comments_df['sentiment'] == 'positive'])
negative_comments_count = len(comments_df[comments_df['sentiment'] == 'negative'])

positive_percentage = (positive_comments_count / total_comments) * 100
negative_percentage = (negative_comments_count / total_comments) * 100

# Find the most liked positive and negative comments
most_liked_positive_comment = comments_df[comments_df['sentiment'] == 'positive'].sort_values(by='like_count', ascending=False).iloc[0]
most_liked_negative_comment = comments_df[comments_df['sentiment'] == 'negative'].sort_values(by='like_count', ascending=False).iloc[0]

# Identify key themes
def extract_key_themes(comments):
    words = ' '.join(comments).split()
    common_words = Counter(words).most_common(10)
    return common_words

positive_comments = comments_df[comments_df['sentiment'] == 'positive']['preprocessed_text']
negative_comments = comments_df[comments_df['sentiment'] == 'negative']['preprocessed_text']

positive_themes = extract_key_themes(positive_comments)
negative_themes = extract_key_themes(negative_comments)

# Summarize feedback
positive_summary = "Main positive feedback themes:\n" + "\n".join([f"{word}: {count}" for word, count in positive_themes])
negative_summary = "Main negative feedback themes:\n" + "\n".join([f"{word}: {count}" for word, count in negative_themes])

# Display results
print(f"Positive comments: {positive_percentage:.2f}%")
print(f"Negative comments: {negative_percentage:.2f}%\n")

print("Most liked positive comment:")
print(f"Comment: {most_liked_positive_comment['comment']}")
print(f"Likes: {most_liked_positive_comment['like_count']}\n")

print("Most liked negative comment:")
print(f"Comment: {most_liked_negative_comment['comment']}")
print(f"Likes: {most_liked_negative_comment['like_count']}\n")

Positive comments: 25.00%
Negative comments: 75.00%

Most liked positive comment:
Comment: The funny part is... Outside India, kids aren't even allowed to give cerelac before 1yr of age by doctors. And we give that to children from 3months in india.
Likes: 2

Most liked negative comment:
Comment: At 9:13, I made a mistake. Sugar is mentioned in the label. But Hidden sugar is not mentioned, which is why the total sugar on the label is incorrect. Read this report for the details
https://stories.publiceye.ch/nestle-babies/

🚨 Unacademy Manzil 2024 - The Biggest Event for UPSC | Join Me LIVE on Unacademy Unstoppable  Event Link: https://www.youtube.com/watch?v=2tzmZ-3Ne3c
 Block Your Calendar | 28 April | 01 PM
Likes: 762

